In [3]:
from transformers import pipeline
import requests
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
# Sample API URL
api_url = "http://127.0.0.1:8000/get-news?stock=Amazon"

# Fetch the news data from the API
response = requests.get(api_url)

# Parse the JSON response
news_data = response.json()

# Function to calculate the average sentiment score
def calculate_average_sentiment(news_list):
    total_score = 0
    count = 0

    for news in news_list:
        # Get the sentiment analysis result for each headline
        result = sentiment_pipeline(news)
        
        # Extract the sentiment score (positive = 1, negative = -1, neutral = 0)
        sentiment = result[0]['label']
        score = result[0]['score']

        # Adjust sentiment score based on label (assumes positive, neutral, negative)
        if sentiment == 'NEGATIVE':
            score = -score

        total_score += score
        count += 1

    if count == 0:
        return 0  # Avoid division by zero

    return total_score / count

# Iterate over the news sources and calculate the average sentiment score
for source, news_list in news_data.items():
    # Check if there are actual news articles and not error messages
    if isinstance(news_list, list) and news_list and "Error" not in news_list[0]:
        avg_sentiment = calculate_average_sentiment(news_list)
        print(f"Average sentiment of news from {source}: {avg_sentiment:.2f}")
    else:
        print(f"No valid news found for {source} or encountered an error.")

Average sentiment of news from AlphaVantage: 0.13
Average sentiment of news from CNBC: -0.19
No valid news found for Marketaux or encountered an error.
Average sentiment of news from NewsAPI: -0.44
Average sentiment of news from Reuters: -0.99
